In [1]:
%load_ext autoreload
%autoreload 2

# ATM Features

In [2]:
import pandas as pd
import numpy as np

from pysal.lib.cg import KDTree, RADIUS_EARTH_KM

import warnings
warnings.simplefilter('ignore')

/Users/renier.botha/anaconda3/envs/mobile_money/lib/python3.7/site-packages/pysal/model/spvcm/abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [3]:
train_df = pd.read_csv('../../data/raw/training.csv', index_col=0)
train_df.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8_1,Q8_2,Q8_3,...,Q17,Q18,Q19,Latitude,Longitude,mobile_money,savings,borrowing,insurance,mobile_money_classification
ID,,,,,,,,,,,,,,,,,,,,,
5086,98,2,3,1,1,2,2,0,0,0,...,-1,4,4,-4.460442,29.811396,0,0,0,0,0
1258,40,1,1,3,5,1,1,1,0,0,...,4,1,4,-6.176438,39.244871,1,1,1,0,3
331,18,2,4,6,3,2,1,0,0,0,...,-1,1,1,-6.825702,37.652798,1,0,0,0,2
6729,50,1,1,3,1,1,1,0,0,0,...,-1,1,4,-3.372049,35.808307,1,0,1,0,3
8671,34,1,1,1,1,2,1,0,1,0,...,-1,1,4,-7.179645,31.039095,1,1,0,1,3


In [4]:
atm_df = pd.read_csv('../../data/raw/FSDT_FinAccessMapping/atm_for_upload_win.csv')
atm_df['yr_operation_started'] = pd.to_datetime(atm_df['yr_operation_started'])
atm_df['years_old'] = 2019 - atm_df['yr_operation_started'].dt.year
atm_df.head()

,region,district,ward,latitude,longitude,onsite_offsite,bank_type,bank_name,bank_name_short,yr_operation_started,number_atms,services_deposits,services_withdrawls,services_bill_payment,services_buy_airtime,other_services,visa_mastercard,umoja_nmb,years_old
0,Dar es Salaam,Ilala,Kariakoo,-6.82175,39.28000,On-site,Commercial Bank ATM,Accessbank (Tanzania) Limited,Access,2003-11-12,1,NaN,Yes,NaN,NaN,NaN,Both,,16
1,Dar es Salaam,Ilala,Kariakoo,-6.81967,39.27322,On-site,UMOJA SWITCH ATM,Accessbank (Tanzania) Limited,Access,2014-09-01,1,NaN,Yes,NaN,NaN,NaN,VISA,,5
2,Dar es Salaam,Kinondoni,Manzese,-6.80376,39.26055,On-site,Commercial Bank ATM,Accessbank (Tanzania) Limited,Access,2010-07-05,1,NaN,Yes,NaN,NaN,NaN,VISA,,9
3,Dar es Salaam,Kinondoni,Makumbusho,-6.77668,39.24587,On-site,Commercial Bank ATM,Accessbank (Tanzania) Limited,Access,2011-11-17,1,NaN,Yes,NaN,NaN,NaN,Mastercard,,8
4,Tabora,Tabora Urban,Kanyenye,-5.02275,32.80941,On-site,Commercial Bank ATM,Accessbank (Tanzania) Limited,Access,2013-12-18,2,Yes,Yes,Yes,Yes,NaN,Both,,6


In [5]:
atm_df.onsite_offsite.unique()

array(['On-site', 'Off-site'], dtype=object)

## Distance to Closest ATM

In [6]:
# Create tree
atm_tree = KDTree(atm_df[['latitude','longitude']].values, distance_metric='ARC', radius=RADIUS_EARTH_KM)

### Prototype with a few points

Randomly select a sample from training, calculate closest atm and then show on the map

In [7]:
import folium
# Select a random coordinate
test_coords = train_df.iloc[np.random.randint(0, len(train_df))][['Latitude','Longitude']]

dist, index = atm_tree.query(test_coords)

# Get coords of closest atm
atm_coords = tuple(atm_df.iloc[index][['latitude','longitude']])


# Create map
m = folium.Map(location=test_coords, height="80%")

# Train marker
folium.CircleMarker(test_coords, radius=10, popup='train',fill=True, fill_color='red', line_color='red').add_to(m)

# ATM marker
folium.CircleMarker(atm_coords, radius=10, popup='ATM').add_to(m)


m

I ran the cell above a couple of times and it seems to be working. 

## Create Features

Functions return a dictionary to `atm_features` method, which combines everything into a series. 

In [8]:
from tqdm import tqdm
tqdm.pandas()

In [9]:
train_coords = train_df[['Latitude','Longitude']]

def to_int(x):
    return np.int16(bool(x))

def closest_atm_features(atm_tree, coords):
    dist, indx = atm_tree.query(coords)
    
    closest_atm = atm_df.iloc[indx]
    
    features = {
        'atm_dist_closest': dist,
        'atm_deposits': closest_atm['services_deposits'],
        'atm_withdrawals': closest_atm['services_withdrawls'],
        'atm_bill_payment': closest_atm['services_bill_payment'],
        'atm_airtime': closest_atm['services_buy_airtime'],
#         'atm_bank_type': closest_atm['bank_type'],
        'atm_years_old': closest_atm['years_old']
    }
    
    return features

def num_atms_in_radius(atm_tree, coords, radius=5):
    
    # Get the closest atms
    closest = atm_tree.query_ball_point(coords,r=radius)
    
    # Number of closest
    n_closest = len(closest)
    
    features = {
        'atm_n_closest': n_closest
    }
    
    return features

def atm_features(atm_tree, coords, **kwargs):
    closest_features = closest_atm_features(atm_tree, coords)
    
    n_closest_features = num_atms_in_radius(atm_tree, coords, **kwargs)
    
    all_features = {**closest_features, **n_closest_features}

    return pd.Series(all_features).replace({'Yes': 1, np.nan: -1})

In [10]:
test = atm_features(atm_tree, test_coords)
test

atm_dist_closest    32.531042
atm_deposits        -1.000000
atm_withdrawals      1.000000
atm_bill_payment    -1.000000
atm_airtime         -1.000000
atm_years_old        6.000000
atm_n_closest        0.000000
dtype: float64

In [11]:
# Test 
test = train_coords.head()
test.progress_apply(lambda coords: atm_features(atm_tree, coords, radius=5), axis=1)

100%|██████████| 5/5 [00:00<00:00, 296.21it/s]


,atm_dist_closest,atm_deposits,atm_withdrawals,atm_bill_payment,atm_airtime,atm_years_old,atm_n_closest
ID,,,,,,,
5086,45.146796,-1.0,1.0,-1.0,-1.0,7.0,0.0
1258,2.219941,-1.0,1.0,-1.0,-1.0,9.0,4.0
331,1.004069,-1.0,1.0,-1.0,-1.0,17.0,21.0
6729,6.278241,-1.0,1.0,-1.0,-1.0,14.0,0.0
8671,30.747110,-1.0,1.0,1.0,1.0,10.0,0.0


## Extract Features and Save to Disk

For both training and testin datasets, extract features and save under `data/modelling/{train/test}_atm_features.csv` 

### Train Features

In [12]:
train_atm_features = train_coords.progress_apply(lambda coords: atm_features(atm_tree, coords, radius=5), axis=1)
train_atm_features.head()

100%|██████████| 7094/7094 [00:16<00:00, 430.22it/s]


,atm_dist_closest,atm_deposits,atm_withdrawals,atm_bill_payment,atm_airtime,atm_years_old,atm_n_closest
ID,,,,,,,
5086,45.146796,-1.0,1.0,-1.0,-1.0,7.0,0.0
1258,2.219941,-1.0,1.0,-1.0,-1.0,9.0,4.0
331,1.004069,-1.0,1.0,-1.0,-1.0,17.0,21.0
6729,6.278241,-1.0,1.0,-1.0,-1.0,14.0,0.0
8671,30.747110,-1.0,1.0,1.0,1.0,10.0,0.0


In [13]:
train_atm_features.to_csv('../../data/modelling/train/atm_features.csv')

### Test Features

In [14]:
test_df = pd.read_csv('../../data/raw/test.csv', index_col=0)

test_coords = test_df[['Latitude','Longitude']]

test_atm_features = test_coords.progress_apply(lambda coords: atm_features(atm_tree, coords, radius=5), axis=1)

test_atm_features.head()

100%|██████████| 2365/2365 [00:05<00:00, 447.75it/s]


,atm_dist_closest,atm_deposits,atm_withdrawals,atm_bill_payment,atm_airtime,atm_years_old,atm_n_closest
ID,,,,,,,
2352,1.559570,1.0,-1.0,-1.0,-1.0,8.0,5.0
8208,30.872633,-1.0,1.0,-1.0,-1.0,6.0,0.0
2785,12.369680,-1.0,1.0,-1.0,-1.0,16.0,0.0
2967,19.112857,-1.0,1.0,-1.0,-1.0,8.0,0.0
1697,0.382182,-1.0,1.0,-1.0,-1.0,8.0,10.0


In [15]:
test_atm_features.to_csv('../../data/modelling/test/atm_features.csv')